# Programming Assignment 2: Support Vector Machine

## Remarks:
1. Please complete the theoretical questions directly in this notebook using `markdown`. We do not accept handwritten PA submissions.
2. You will use `cvxopt` package in this assignment. To ensure consistency of notation, please use `pip install cvxopt==1.2.7` to install a suitable version (1.3.0 version may encounter problems)
3. You will use `torch` to implement binary classification and `torchvision` to load MNIST dataset. `tqdm` is used to show the learning progress.

In [ ]:
! pip install cvxopt==1.2.7
! pip install scikit-learn
! pip install torch torchvision tqdm

## Recap of Soft-margin SVM Optimization Problem
A SVM computes a hyperplane classifier of the form $f(x)=w^{T}x+b$, where $w$ is the normal vector to the hyperplane, $x$ is the input vector and $b$ is the bias scalar. Since we apply it to a binary classification problem, we will ultimately predict label $y = 1$ if $f(x) ≥ 0$ and $y = −1$ if $f(x) < 0$. By deriving the dual form of the SVM problem, we convert the original non-convex problem into a Quadratic Programming (QP) problem over a set of Lagrange multipliers $\alpha=\{ \alpha_1,\alpha_2,\ldots,\alpha_N\}$ . 

Consider a dataset with $N$ samples $\{x_1,x_2,\ldots,x_N\}$. The soft-margin SVM algorithm tries to solve:

$$\begin{array}{rc}\underset{\alpha}{max} &\sum\limits^N_{i=1}\alpha_i-\frac{1}{2}\sum\limits^N_{i=1}\sum\limits^N_{j=1}\alpha_i\alpha_jy_iy_jK(x_i,x_j) \\ \text { s.t. } & \sum\limits^N_{i=1}\alpha_iy_i=0, \\ & 0 \leqslant \alpha_i \leqslant C,i=1,2,\ldots,N.\end{array}$$

where $C$  is the penalty parameter. $K$ is the kernel function, which allow us to learn non-linear decision function through feature mapping. When an inner product is used (e.g. $K(x_i,x_j)=\langle x_i,x_j\rangle$ ), the model degenerates into a linear SVM. We can also represent the kernel function as a symmetric matrix $K_{i,j} = K(x_i, x_j)$. 

After finding the optimal solution $\alpha^{*}$ , we could obtain the optimal $w^*$ and $b^*$ and then derive the decision function for classification: 
$f(x)=\sum\limits_{i=1}^{N} \alpha_{i}^{*} y_{i} K\left(x, x_{i}\right)+b^{*}.$ The KKT conditions can be used to check for convergence to the optimal solution. In other words, any $\alpha$ that satisfy these properties will be an optimal solution to the
optimization problem given above. For this problem the KKT conditions are:
$$
\begin{aligned}
\alpha_i=0 & \Rightarrow y^{(i)}\left(w^T x^{(i)}+b\right) \geq 1 \\
\alpha_i=C & \Rightarrow y^{(i)}\left(w^T x^{(i)}+b\right) \leq 1 \\
0<\alpha_i<C & \Rightarrow y^{(i)}\left(w^T x^{(i)}+b\right)=1.
\end{aligned}
$$

In this assignment, we will introduce two different ways of solving $\alpha^{*}$.

## Part I: SVM as a Quadratic Programming Problem

### Solving QP Problem with CVXOPT Package

In this section, we will use a Python package `cvxopt` to see how SVM works. The `cvxopt` is widely used in solving optimization problems. We are also going to use `numpy`, `pandas`, `random`, `seaborn` and `matplotlib` packages to help us. The first thing we need to do is to import them.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cvxopt
from cvxopt import matrix, solvers
from sklearn import datasets

%matplotlib inline

QP solves quadratic optimization problems, to optimize a multivariate quadratic function subject with linear constraints on the variables. They can be solved via the solvers.qp() function in `cvxopt`. The standard form of a QP (following CVXOPT notation) is:

$$\begin{array}{rc}\underset{x}{min}  & \frac{1}{2} x^{\mathrm{T}} P x+q^{\mathrm{T}} x \\ \text { s.t. } & G x \preceq h, \\ & A x=b.\end{array}$$

#### Q1. Before using it to solve SVM, we need to transform the soft-margin SVM optimization problem above into the standard form. Please derive corresponding $P,q,G,h,A,b$ here and show the derivation steps.

*Your Answer:*
$$P=\\ q= \\ G= \\ h= \\ A= \\ b= $$
Derivation:

#### Q2. Now use `numpy` and `cvxopt` to implement soft-margin SVM algorithm in the form of QP problem. Check this document for its usage: https://cvxopt.org/examples/tutorial/qp.html. (Remark: the data type of matrix objects for qp function should be double)

In [4]:
def linear_kernel(x1,x2):
    return np.dot(x1,x2)

def kernel_matrix(X, n_samples, kernel):
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = kernel(X[i], X[j])
    return K

In [5]:
def svm_qp(X, y, C , kernel=linear_kernel):
    # X: feature matrix, X[i,:] is the vector x_i
    # y: labels
    # C: soft-margin parameter
    # kernel: kernel function. default linear kernel
    # return optimal alpha and b of SVM

    N, n_features = X.shape
    K = kernel_matrix(X, N, kernel)
    """ Your Code here """
    P = 
    q = 
    G = 
    h = 
    A = 
    b = 
    """ Your Code here """

    # Solve Quadratic Programming problem:
    solvers.options['maxiters'] = 200
    solution = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(solution['x']).reshape(N) # results of qp
    # Remark: In the result, some alphas that are infinitely close to zero in value should be considered as 0.
    # You can set a threshold for that, e.g. 1e-4.
    alphas[alphas<1e-4] = 0
    # Calculate bias b in svm
    """ Your Code here """
    b_svm = 
    """ Your Code here """

    return alphas, b_svm

In [6]:
def f(data, X, y, alphas, b, kernel):
    # return decision function f(x) on vector point x=`data`
    # X is feature matrix and y is the label list
    # alphas and b are SVM outputs
    """ Your Code here """
    
    """ Your Code here """

Now you have implemented the SVM algorithm. Let's test it on a small dataset and plot the hyperplane for visulization of support vectors.

In [ ]:
def plot(X, y, alphas, b, kernel):
    # visulize the decision hyperplane on the dataset
    x_min = min(X[:, 0]) - 0.5 # get boundry of the figure, x y here are coordinate axis
    x_max = max(X[:, 0]) + 0.5
    y_min = min(X[:, 1]) - 0.5
    y_max = max(X[:, 1]) + 0.5
    step = 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))
    d = np.array([[x1, x2] for x1, x2 in zip(np.ravel(xx), np.ravel(yy))])
    Z = np.zeros(len(d))
    for i in range(len(d)): 
        Z[i] = f(d[i], X, y, alphas, b, kernel)
    Z = Z.reshape(xx.shape)

    fig, ax = plt.subplots()
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, palette='winter') # draw the data points
    ax.contour(xx, yy, Z, levels=[-1, 0, 1]) # draw the hyperplane
    plt.show()
    
X = np.array([[3, 4], [1, 4],[2, 3], [6, -1], [7, -1], [5, -3], [2, 4]])
y = np.array([-1, -1, -1, 1, 1, 1, 1])
alphas, b = svm_qp(X, y, C=0.1, kernel=linear_kernel)
plot(X, y, alphas, b, kernel=linear_kernel)

#### Q3. While the data can be classified by soft-margin SVM, we still don't know which parameter can realize the "best" performance. Run with different penalty parameters $C$ and try to find an "optimal" value. Compare and analyze you results, e.g. why it is an optimal parameter, what is happening as $C$ is getting larger(smaller).

#### Q4. When the data set is not linearly separable, we need to utilize different kernel functions. Here is a non-linear dataset, please test different kernel functions (e.g. Polynomial kernel and RBF kernel) and choose one to fit a soft-margin SVM model on it. 

In [ ]:
# Generate half moon data (ref: https://shadow-ssml.readthedocs.io/en/latest/examples/halfmoons_example.html)
# parameters for data generation, you may modify them for further observation and comparison
n_samples = 100  # number of samples to generate
noise = 0.1  # noise to add to sample locations
X, y = datasets.make_moons(n_samples=n_samples, noise=noise)
y = (y - 0.5) * 2

def your_kernel(x1, x2):
    """ Your Code here """
    pass
    """ Your Code here """

alphas, b = svm_qp(X, y, C=10, kernel=your_kernel)
plot(X, y, alphas, b, kernel=your_kernel)

## Part II: Hinge Loss for Binary Classification

Beside SVM, here we also introduce Hinge Loss, which formulates the binary classification loss as:
\begin{equation}
    L(y) = max(0, c - y \cdot \hat y),
\end{equation}
where $c$ is a hyperparameter (usually $1$), $y$ is the true label ($\pm 1$) and $\hat y$ is the prediction ([-1, 1]). As the predictions get further and further away from the true labels, the loss also gets larger. Additionally, once the prediction is correct ($y \cdot \hat y$), the loss is $0$ to avoid overconfidence. 

Actually, using Hinge Loss for binary classfication is equivalent to soft-margin SVM by considering an L2 regularization. The problem of using Hinge Loss for binary classification can be formulated as:
$$
   \text{minimize} \frac{1}{N} \sum_i max(0, 1 - y_i(W^T x_i + b)) + \lambda ||W||^2.
$$
If you are familiar with soft-margin SVM, you will easily find the equivalence by denoting  $ \zeta_i = max(0, 1 - y_i(W^T x_i + b))$:
$$
   \text{minimize} \frac{1}{N} \sum_i \zeta_i + \lambda ||W||^2, \\
   \text{subject to} \quad \zeta_i > 0,\quad  1 - y_i(W^T x_i + b) \le \zeta_i \quad \text{for all $i$},
$$
which is exactly the formulation of soft-margin SVM.


In this section, we will use Hinge Loss as the loss function of Stochastic Gradient Descent (SGD) method to classfy 0 and 1 in MNIST. (If you are not familiar with the mechanism of auto differentiation, you can refer to https://d2l.ai/chapter_preliminaries/autograd.html)

First we need to load MNIST dataset.

In [ ]:
import torch
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt

device =  "cpu" # we will use cpu in this assignment
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(mean = [0.5],std = [0.5])])
path = './data/'
TRAIN_DATA = torchvision.datasets.MNIST(path, train = True,transform = transform, download = True)
VAL_DATA = torchvision.datasets.MNIST(path, train = False,transform = transform, download = True)

Only data labelled as 0 and 1 is needed, so we preprocess data with the following function:

In [3]:
def prepare_data(data, label, num_samples):
    # Filter the dataset to include only the specified labels
    # data: the original dataset
    # label(list): needed labels, e.g. [0, 1]
    # num_samples(list): samples for each label
    
    indices = [i for i, (img, lbl) in enumerate(data) if lbl in label]
    filtered_data = torch.utils.data.Subset(data, indices)
    
    # Further reduce the dataset to the specified number of samples per label
    label_counts = {lbl: 0 for lbl in label}
    selected_indices = []
    
    for i in indices:
        _, lbl = data[i]
        if label_counts[lbl] < num_samples[label.index(lbl)]:
            selected_indices.append(i)
            label_counts[lbl] += 1
        if all(count >= num_samples[label.index(lbl)] for count in label_counts.values()):
            break
    
    final_data = torch.utils.data.Subset(data, selected_indices)
    
    # Create DataLoader for the preprocessed data
    data_loader = torch.utils.data.DataLoader(final_data, batch_size=1, shuffle=True)
    
    return final_data, data_loader

train_data, train_data_loader = prepare_data(TRAIN_DATA, label=[0,1], num_samples=[200,200])
val_data, val_data_loader = prepare_data(VAL_DATA, label=[0,1], num_samples=[50,50])

#### Q5: Then we need to implement Hinge Loss in Equation(1) function with `Torch`.

In [5]:
def hinge_loss(preds, targets, c=1):
    # calculate hinge loss with torch tensor
    # preds: Batch_size x 1, predicted scores
    # targets: Batch_size x 1, ground truth labels
    # loss: Real number, hinge loss for preds and targets
    """ Your Code here """
    loss=
    """ Your Code here """
    return loss

#### Q6(Optional): Finally implement the training loop. (It might take a minute for training.)

In [ ]:
# some hyperparameters
lr=1e-4
epochs=80

# initialize linear model
# requires_grad = True to make w and b trainable
w=torch.randn(784,1,requires_grad=True)
b=torch.randn(1,requires_grad=True)

# optimizer
optimizer=torch.optim.SGD([w,b],lr=lr)

losses=[]
accuracy=[]
# training
for epoch in tqdm(range(epochs)):
    avg_loss = 0
    for i,(img,lbl) in enumerate(train_data_loader):
        # forward propogate
        # calulate the predicted scores and loss
        # You may use the hinge_loss function in Q5
        # hint: to ensure the robustness of the model, you may add L2 regularization term to the loss function 
        # 
        """ Your Code here """
        loss=
        """ Your Code here """
        # backward propogate
        # update the weights and bias using torch optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss+=loss.item()
    # record average loss at each epoch
    avg_loss/=len(train_data_loader)
    losses.append(avg_loss)
    
    # validation
    # test the model on validation set and calculate the accuracy at each epoch
    avg_accu = 0
    # When testing, we need to shut requires_grad_ down to prevent the model being trained on the validation set
    w.requires_grad_(False)
    b.requires_grad_(False)
    # torch.no_grad shuts down the automatic calculation of gradients
    with torch.no_grad():
        for i,(img,lbl) in enumerate(val_data_loader):
            img=img.view(-1,784)
            preds=torch.matmul(img,w)+b
            preds = (preds.squeeze() > 0).int()
            accu = (preds == lbl).float().mean()  
            avg_accu += accu.item()
    # record the testing accuracy at each epoch 
    avg_accu /= len(val_data_loader)    
    accuracy.append(avg_accu)
    # after validation, we need to make w and b trainable again
    w.requires_grad_(True)
    b.requires_grad_(True)

# plot the training loss and validation accuracy
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.subplot(1,2,2)
plt.plot(accuracy)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.show()

#### Q7(Optional): Replace Hinge Loss with [Binary Cross-Entropy(BCE) Loss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) and compare the results.